# Model calibration

In this tutorial, we will calibrate the free parameters of PPI's model. For this, we will load the source file containing the model's code and the auxiliary functions with the calibration algorithm.

## Importing Python's libraries to manipulate and visualise data

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

## Importing PPI's functions

In this example, we will import the PPI source code directly from the repository. This means that we will place a request to GitHub, download the `ppi.py` file, and copy it locally into the folder where these tutorials are saved. Then, we will import ppi.

In [2]:
import requests
url = 'https://raw.githubusercontent.com/oguerrer/ppi/main/source code/ppi.py'
r = requests.get(url)
with open('ppi.py', 'w') as f:
    f.write(r.text)
import ppi

## Load data

Now we load the same dataset that we explored in the previous tutorial.

### Indicators

### Interdependency network

### Budget

### Budget-indicator mapping